# Nim (21 Sticks Variant)

## Setup
- Start with **21 sticks** (or counters, stones, matches, etc.).  
- Two players take turns.  

---

## Rules
1. On your turn, you must take **1, 2, or 3 sticks** from the pile.  
2. Players alternate turns.  
3. **The player forced to take the last stick loses.**  

---

## Example Play
- Start: 21 sticks.  
- Player A takes 2 → 19 left.  
- Player B takes 3 → 16 left.  
- Player A takes 1 → 15 left.  
- … and so on, until one player is forced to take the last stick and loses.  



In [1]:
from Game import *

Version:  0.3.15


In [2]:
def initial_state():
    return 21

In [3]:
def show_state(state,player):
    print("Player",player)
    print(f"Sticks remaining: {state}")

In [4]:
def valid_moves(state,player):
    # return a **list** of moves that are valid

    if state==1:
        return [1]
    elif state==2:
        return [1,2]
    else:
        return [1,2,3]
    

In [5]:
def update_state(state,player,move):
    new_state = state - move
    return new_state

In [6]:
def win_status(state,player):
    # return None if the game is not over
    # return 'win' if player has won
    # return 'lose' if player has lost
    # return 'stalemate' if player has stalemate

    if player==1:
        other_player=2
    else:
        other_player=1

    if state==0:
        return 'lose'
    else:
        return None

## Agents

In [7]:
def human_move(state,player):
    move = int(input("Enter your move (1, 2, or 3): "))
    while move not in valid_moves(state,player):
        print("Invalid move. Try again.")
        move = int(input("Enter your move (1, 2, or 3): "))
    return move

human_agent=Agent(human_move)

In [8]:
def random_move(state,player):
    return random.choice(valid_moves(state,player))

random_agent=Agent(random_move)

In [9]:
from Game.minimax import *
def minimax_move(state,player):
    values,actions = minimax_values(state,player,display=False)
    return top_choice(actions,values)
minimax_agent=Agent(minimax_move)

## Skittle Agent

In [10]:
def skittles_move(state,player,info):
    T=info.T   # table for the skittles
    learning=info.learning
    last_state=info.last_state
    last_action=info.last_action
    

    # if we haven't seen this state before, initialize it
    if state not in T:
        actions=valid_moves(state,player)
        T[state]=Table()
        for action in actions:
            T[state][action]=2  # start with 1 skittle for each action

    move=weighted_choice(T[state])

    if move is None:
        move=random_move(state,player)

        if learning:
            if last_state:
                T[last_state][last_action]-=1  # punish last action == remove one skittle
                if T[last_state][last_action]<0:
                    T[last_state][last_action]=0  # don't go below zero
    
    return move


def skittles_after(status,player,info):
    T=info.T   # table for the skittles
    learning=info.learning
    last_state=info.last_state
    last_action=info.last_action

    if learning:
        if status=='lose':
            T[last_state][last_action]-=1  # punish last action == remove one skittle
            if T[last_state][last_action]<0:
                T[last_state][last_action]=0  # don't go below zero


In [11]:
skittles_agent=Agent(skittles_move)
skittles_agent.T=Table()  # starts off empty
skittles_agent.post=skittles_after
skittles_agent.learning=True

## Running the Game

In [12]:
g=Game()
g.run(minimax_agent,skittles_agent)

====
Game  1
Player 1
Sticks remaining: 21
Player 1 moves 3
Player 2
Sticks remaining: 18
Player 2 moves 2
Player 1
Sticks remaining: 16
Player 1 moves 3
Player 2
Sticks remaining: 13
Player 2 moves 2
Player 1
Sticks remaining: 11
Player 1 moves 2
Player 2
Sticks remaining: 9
Player 2 moves 1
Player 1
Sticks remaining: 8
Player 1 moves 3
Player 2
Sticks remaining: 5
Player 2 moves 2
Player 1
Sticks remaining: 3
Player 1 moves 2
Player 2
Sticks remaining: 1
Player 2 moves 1
Player 2
Sticks remaining: 0
Player  1 won.


[1]

In [15]:
skittles_agent.stuff

{'move': <function __main__.skittles_move(state, player, info)>,
 'update': <function Game.game.no_update(observation, action, reward, done, memory)>,
 'memory': {},
 'T': {18: {1: 2, 2: 2, 3: 2},
  13: {1: 2, 2: 2, 3: 2},
  9: {1: 2, 2: 2, 3: 2},
  5: {1: 2, 2: 2, 3: 2},
  1: {1: 1}},
 'post': <function __main__.skittles_after(status, player, info)>,
 'learning': True,
 'states': [18, 13, 9, 5, 1],
 'actions': [2, 2, 1, 2, 1],
 'last_action': 1,
 'last_state': 1,
 'last_player': 2,
 'move_args': 3,
 'move_count': 10}

In [16]:
skittles_agent.move_count

10

In [29]:
g=Game(number_of_games=500)
g.display=False
result=g.run(minimax_agent,skittles_agent)
print(result)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [30]:
skittles_agent.T

{18: {1: 2, 2: 0, 3: 0},
 16: {1: 0, 2: 0, 3: 2},
 13: {1: 0, 2: 0, 3: 0},
 9: {1: 0, 2: 0, 3: 0},
 5: {1: 0, 2: 0, 3: 0},
 1: {1: 0},
 20: {1: 0, 2: 0, 3: 2},
 17: {1: 0, 2: 0, 3: 0},
 14: {1: 2, 2: 0, 3: 0},
 19: {1: 0, 2: 2, 3: 0},
 15: {1: 0, 2: 2, 3: 0},
 10: {1: 2, 2: 0, 3: 0},
 7: {1: 0, 2: 2, 3: 0},
 12: {1: 0, 2: 0, 3: 2},
 11: {1: 0, 2: 2, 3: 0},
 2: {1: 2, 2: 0},
 3: {1: 0, 2: 2, 3: 0},
 8: {1: 0, 2: 0, 3: 2},
 6: {1: 2, 2: 0, 3: 0},
 4: {1: 0, 2: 0, 3: 2}}

In [31]:
SaveTable(skittles_agent.T,'Nim21_Skittles.json')